This notebook assumes we have 2 users top spotify data, and experiments with how we can compare them, and utilize them to build a new playlist.

First we grab our SpotifyAPI class, so when we need to make API calls, we can

In [36]:
import base64
import datetime
from urllib.parse import urlencode

import requests

#Follows Spotify's "Implicit Grant" Authorization flow
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    authorization_url = "https://accounts.spotify.com/authorize"
    
    #external functions ("public")
    

    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')

    def get_playlist(self, _id):
        return self.get_resource(_id, resource_type='playlists')

    def get_playlist_tracks(self, _id):
        return self.get_resource(_id, resource_type='playlists', addendum='tracks')

    def get_track_features(self, _id):
        return self.get_resource(_id, resource_type='audio-features')

    def get_track_analysis(self, _id):
      return self.get_resource(_id, resource_type='audio-analysis') 

    def search(self, query=None, operator=None, operator_query=None, search_type='artist', limit=20 ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower(), "limit": str(limit)})
        return self.base_search(query_params)

    #generic api request, accepts the full endpoint string

    def generic_get(self, endpoint):
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            print(r.status_code)
            if r.status_code == 429:
              print('Retry after ' + str(r.headers['Retry-After']) + ' seconds')
              raise Exception("You are requesting too much data, retry after " + str(r.headers['Retry-After']) + ' seconds')
            return {}
        return r.json()

    def generic_post(self, endpoint):
        headers = self.get_resource_header()
        r = requests.post(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            print(r.status_code)
            return {}
        return r.json()

    def generic_put(self, endpoint):
        headers = self.get_resource_header()
        r = requests.put(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            print(r.status_code)

            
    #internal functions ("private")


    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1', addendum=''):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}/{addendum}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()

    #new functions for user functions

    def get_current_user(self):
        endpoint = f"https://api.spotify.com/v1/me"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

    def get_current_user_recently_played(self):
        endpoint = f"https://api.spotify.com/v1/me/player/recently-played"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

    def get_current_user_top(self, top_type='tracks', time_range='medium_term'):
        #endpoint = f"https://api.spotify.com/v1/me/top/{top_type}?limit=50&time_range{time_range}"
        endpoint = f"https://api.spotify.com/v1/me/top/{top_type}?time_range={time_range}&limit=50"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            print(r.status_code)
            return {}
        return r.json()

    def get_user(self, user):
        endpoint = f"https://api.spotify.com/v1/users/{user}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()


Now some other initializations that we have covered in previous notebooks.

In [65]:
import webbrowser
import json

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import time

In [38]:
#Initializing spotify API and defining some important constants
client_id = ""
client_secret = ""

spotify = SpotifyAPI(client_id, client_secret)
indices = {'resource_types':['tracks','artists'],'time_frames':['short_term','medium_term', 'long_term']}

Now for the good stuff. We load in each user's data

In [39]:
#Uploading json files containing user data
from google.colab import files
uploaded = files.upload()

Saving kyle.sellers (2).json to kyle.sellers (2) (1).json
Saving phillwill16.json to phillwill16 (1).json


In [40]:
!ls

'kyle.sellers (2) (1).json'  'phillwill16 (1).json'   sample_data
'kyle.sellers (2).json'       phillwill16.json


In [41]:
#get all user data raw from json files
user_1_file_name = ''
user_2_file_name = ''

def getUserTopDataFromJson(file_name):
  with open(file_name) as json_file:
    return json.load(json_file)

user_1_data = getUserTopDataFromJson(user_1_file_name)
user_2_data = getUserTopDataFromJson(user_2_file_name)

print(user_1_data)
print(user_2_data)

{'tracks_short_term': [{'album': {'album_type': 'ALBUM', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/36PWgrlKrQpL8vvRR2kURR'}, 'href': 'https://api.spotify.com/v1/artists/36PWgrlKrQpL8vvRR2kURR', 'id': '36PWgrlKrQpL8vvRR2kURR', 'name': 'Origami Button', 'type': 'artist', 'uri': 'spotify:artist:36PWgrlKrQpL8vvRR2kURR'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'US', 'UY', 'VN', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/5SjcJ9pVDkFI5swmlEiUWh'}, 'href': 'https://api.spotify.com/v1/albums/5SjcJ9pVDkFI5swmlEiU

In [42]:

def addRanks(user_data):
  for resource_type in indices['resource_types']:
    for time_frame in indices['time_frames']:
      rank = 0
      for resource in user_data[resource_type + '_' + time_frame]:
        rank += 1
        resource['rank'] = rank

addRanks(user_1_data)
addRanks(user_2_data)

for item in user_1_data['artists_long_term']:
  print(item)

for item in user_2_data['artists_long_term']:
  print(item)



{'external_urls': {'spotify': 'https://open.spotify.com/artist/05MlomiA9La0OiNIAGqECk'}, 'followers': {'href': None, 'total': 50006}, 'genres': ['brighton indie', 'british math rock', 'progressive post-hardcore'], 'href': 'https://api.spotify.com/v1/artists/05MlomiA9La0OiNIAGqECk', 'id': '05MlomiA9La0OiNIAGqECk', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5ebef86feb3e2b9c57897ddc453', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174ef86feb3e2b9c57897ddc453', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178ef86feb3e2b9c57897ddc453', 'width': 160}], 'name': 'Delta Sleep', 'popularity': 50, 'type': 'artist', 'uri': 'spotify:artist:05MlomiA9La0OiNIAGqECk', 'rank': 1}
{'external_urls': {'spotify': 'https://open.spotify.com/artist/3mIj9lX2MWuHmhNCA7LSCW'}, 'followers': {'href': None, 'total': 5011490}, 'genres': ['modern alternative rock', 'modern rock', 'pop', 'rock'], 'href': 'https://api.spotify.com

In [43]:
#defining relative weights of time periods, as well as how the #1 is scaled relative to the last in each time period
short_term_weight = 0.3
medium_term_weight = 0.3
long_term_weight = 0.4

short_term_scaling = 3
medium_term_scaling = 3
long_term_scaling = 3

time_frame_weights_scalings = {
    'short_term': {'weight': short_term_weight,'scaling':short_term_scaling},
    'medium_term': {'weight': medium_term_weight,'scaling':medium_term_scaling},
    'long_term': {'weight': long_term_weight,'scaling':long_term_scaling}
}

In [44]:
# #translating rank to a weight for each item
# for resource_type in indices['resource_types']:
#   for time_frame in indices['time_frames']:
#     scaling = time_frame_weights_scalings[time_frame]['scaling']
#     rank = 0
#     for resource in user_1_data[resource_type + '_' + time_frame]:
#       amount_per_step = ((scaling-1)/scaling) / (len(user_1_data[resource_type + '_' + time_frame]) - 1)
#       resource['weight'] = (1/scaling + ((len(user_1_data[resource_type + '_' + time_frame]) - (resource['rank'])) * amount_per_step)) * time_frame_weights_scalings[time_frame]['weight']
#     rank = 0
#     for resource in user_2_data[resource_type + '_' + time_frame]:
#       amount_per_step = ((scaling-1)/scaling) / (len(user_2_data[resource_type + '_' + time_frame]) - 1)
#       resource['weight'] = (1/scaling + ((len(user_2_data[resource_type + '_' + time_frame]) - (resource['rank'])) * amount_per_step)) * time_frame_weights_scalings[time_frame]['weight']






def addWeights(user_data):
  for resource_type in indices['resource_types']:
    for time_frame in indices['time_frames']:
      scaling = time_frame_weights_scalings[time_frame]['scaling']
      rank = 0
      for resource in user_data[resource_type + '_' + time_frame]:
        amount_per_step = ((scaling-1)/scaling) / (len(user_data[resource_type + '_' + time_frame]) - 1)
        resource['weight'] = (1/scaling + ((len(user_data[resource_type + '_' + time_frame]) - (resource['rank'])) * amount_per_step)) * time_frame_weights_scalings[time_frame]['weight']

addWeights(user_1_data)
addWeights(user_2_data)

for item in user_1_data['artists_short_term']:
  print(item['name'] +' : ' + str(item['weight']))

for item in user_2_data['artists_short_term']:
  print(item['name'] +' : ' + str(item['weight']))

Origami Button : 0.3
Properties of Nature : 0.29565217391304344
Wolf & Bear : 0.2913043478260869
Eidola : 0.2869565217391304
Dance Gavin Dance : 0.2826086956521739
Invalids : 0.27826086956521734
TTNG : 0.2739130434782608
Meliorist : 0.26956521739130435
Floral : 0.2652173913043478
Covet : 0.26086956521739124
Good Game : 0.2565217391304348
Tensor : 0.25217391304347825
Their Dogs Were Astronauts : 0.2478260869565217
Colorblind : 0.2434782608695652
Sawce : 0.23913043478260868
Eternity Forever : 0.23478260869565215
Pulses. : 0.2304347826086956
Clairo : 0.22608695652173913
Delta Sleep : 0.22173913043478258
Owane : 0.21739130434782605
WILLOW : 0.21304347826086953
Jank : 0.20869565217391303
Totorro : 0.20434782608695648
Surfaces : 0.19999999999999998
Snakedersch : 0.1956521739130435
Land Wars : 0.19130434782608693
Minus the Bear : 0.18695652173913038
Dammit I'm Mad : 0.18260869565217389
Chon : 0.1782608695652174
Polyphia : 0.17391304347826084
Clever Girl : 0.16956521739130434
Three Trapped Tig

Parsing the data into more developer friendly formats

In [45]:
def getCombinedData(user_data):
  user_combined = {'artists': [], 'tracks': []}
  for resource_type in indices['resource_types']:
    combined_data = []
    for resource in user_data[resource_type + '_short_term']:
      combined_data.append(resource.copy())
    
    redundant_data = []
    for combined_resource in combined_data:
      for resource in user_data[resource_type + '_medium_term']:
        if combined_resource['id'] == resource['id']:
          combined_resource['weight'] += resource['weight']
          redundant_data.append(resource.copy())
    
    for resource in user_data[resource_type + '_medium_term']:
      to_add = True
      for redundant_resource in redundant_data:
        if resource['id'] == redundant_resource['id']:
          to_add = False
      if to_add:
        combined_data.append(resource.copy())
    

    redundant_data = []
    for combined_resource in combined_data:
      for resource in user_data[resource_type + '_long_term']:
        if combined_resource['id'] == resource['id']:
          combined_resource['weight'] += resource['weight']
          redundant_data.append(resource.copy())
    
    for resource in user_data[resource_type + '_long_term']:
      to_add = True
      for redundant_resource in redundant_data:
        if resource['id'] == redundant_resource['id']:
          to_add = False
      if to_add:
        combined_data.append(resource.copy())
    
    user_combined[resource_type] = combined_data
  return user_combined

user_1_combined = getCombinedData(user_1_data)
user_2_combined = getCombinedData(user_2_data)

# print(len(user_1_combined['artists']))
# for val in user_1_combined['artists']:
#   print(val['weight'])
  
for val in user_1_combined['artists']:
  name = val['name']
  weight = val['weight']
  print(f'{name} : {weight}')



  # resources_to_add =  user_1_data[resource_type + '_medium_term'].copy()
    # for redundant_resource in redundant_data:
  # for resource in resources_to_add:
  #   for redundant_resource in redundant_data:
  #     if
  # combined_resource_temp
    
  #for resource in combined_data

Origami Button : 0.7945578231292516
Properties of Nature : 0.7303460514640638
Wolf & Bear : 0.50150842945874
Eidola : 0.5583850931677018
Dance Gavin Dance : 0.9662821650399289
Invalids : 0.4843833185448092
TTNG : 0.7684708666075125
Meliorist : 0.7382727003845015
Floral : 0.49174800354924575
Covet : 0.393522626441881
Good Game : 0.4422360248447205
Tensor : 0.25217391304347825
Their Dogs Were Astronauts : 0.2478260869565217
Colorblind : 0.2434782608695652
Sawce : 0.23913043478260868
Eternity Forever : 0.43682342502218274
Pulses. : 0.8937000887311446
Clairo : 0.22608695652173913
Delta Sleep : 0.8686779059449867
Owane : 0.8384797397219759
WILLOW : 0.21304347826086953
Jank : 0.47332150251404914
Totorro : 0.20434782608695648
Surfaces : 0.19999999999999998
Snakedersch : 0.1956521739130435
Land Wars : 0.19130434782608693
Minus the Bear : 0.18695652173913038
Dammit I'm Mad : 0.18260869565217389
Chon : 0.7694173321502513
Polyphia : 0.5140490979000296
Clever Girl : 0.16956521739130434
Three Trapp

In [46]:
for val in user_1_combined['artists']:
  name = val['name']
  weight = val['weight']
  print(f'{name} : {weight}')


Origami Button : 0.7945578231292516
Properties of Nature : 0.7303460514640638
Wolf & Bear : 0.50150842945874
Eidola : 0.5583850931677018
Dance Gavin Dance : 0.9662821650399289
Invalids : 0.4843833185448092
TTNG : 0.7684708666075125
Meliorist : 0.7382727003845015
Floral : 0.49174800354924575
Covet : 0.393522626441881
Good Game : 0.4422360248447205
Tensor : 0.25217391304347825
Their Dogs Were Astronauts : 0.2478260869565217
Colorblind : 0.2434782608695652
Sawce : 0.23913043478260868
Eternity Forever : 0.43682342502218274
Pulses. : 0.8937000887311446
Clairo : 0.22608695652173913
Delta Sleep : 0.8686779059449867
Owane : 0.8384797397219759
WILLOW : 0.21304347826086953
Jank : 0.47332150251404914
Totorro : 0.20434782608695648
Surfaces : 0.19999999999999998
Snakedersch : 0.1956521739130435
Land Wars : 0.19130434782608693
Minus the Bear : 0.18695652173913038
Dammit I'm Mad : 0.18260869565217389
Chon : 0.7694173321502513
Polyphia : 0.5140490979000296
Clever Girl : 0.16956521739130434
Three Trapp

In [47]:
# take second element for sort
def getWeight(resource):
    return resource['weight']

# sort list with key
user_1_combined['artists'].sort(key=getWeight, reverse=True)

for val in user_1_combined['artists']:
  name = val['name']
  weight = val['weight']
  print(f'{name} : {weight}')

Dance Gavin Dance : 0.9662821650399289
Pulses. : 0.8937000887311446
Delta Sleep : 0.8686779059449867
Owane : 0.8384797397219759
Origami Button : 0.7945578231292516
Chon : 0.7694173321502513
TTNG : 0.7684708666075125
Meliorist : 0.7382727003845015
Properties of Nature : 0.7303460514640638
Intervals : 0.7095829636202307
Sleep Token : 0.6639160011830818
Porter Robinson : 0.6333333333333333
Tyler, The Creator : 0.6129251700680272
Eidola : 0.5583850931677018
Flume : 0.5299319727891156
Polyphia : 0.5140490979000296
Daft Punk : 0.5136054421768708
Lil Uzi Vert : 0.5068027210884354
Kanye West : 0.5054421768707483
Wolf & Bear : 0.50150842945874
Floral : 0.49174800354924575
Invalids : 0.4843833185448092
Jank : 0.47332150251404914
EDEN : 0.44658385093167696
Good Game : 0.4422360248447205
Eternity Forever : 0.43682342502218274
Zedd : 0.43061224489795924
The 1975 : 0.3945578231292517
Covet : 0.393522626441881
Deorro : 0.38299319727891157
Ben Rosett : 0.38299319727891157
Lil Wayne : 0.381632653061224

In [48]:
user_1_combined

{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6guC9FqvlVboSKTI77NG2k'},
   'followers': {'href': None, 'total': 515464},
   'genres': ['metalcore',
    'pixie',
    'pop punk',
    'sacramento indie',
    'screamo',
    'swancore'],
   'href': 'https://api.spotify.com/v1/artists/6guC9FqvlVboSKTI77NG2k',
   'id': '6guC9FqvlVboSKTI77NG2k',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab6761610000e5eb3a80120da477d11601c88b48',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/ab676161000051743a80120da477d11601c88b48',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/ab6761610000f1783a80120da477d11601c88b48',
     'width': 160}],
   'name': 'Dance Gavin Dance',
   'popularity': 69,
   'rank': 5,
   'type': 'artist',
   'uri': 'spotify:artist:6guC9FqvlVboSKTI77NG2k',
   'weight': 0.9662821650399289},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/0tko2stnUOqejIto8rKRh

In [49]:
#
# Begin Pretty Useless things
#

# user_1_tracks_all = []
# user_1_tracks_all.append(user_1_data['tracks_short_term'])
# user_1_tracks_all.append(user_1_data['tracks_medium_term'])
# user_1_tracks_all.append(user_1_data['tracks_long_term'])
# user_1_artists_all = user_1_data['artists_short_term'].append(user_1_data['artists_medium_term'].append(user_1_data['artists_long_term']))
# user_2_tracks_all = user_2_data['tracks_short_term'].append(user_2_data['tracks_medium_term'].append(user_2_data['tracks_long_term']))
# user_2_artists_all = user_2_data['artists_short_term'].append(user_2_data['artists_medium_term'].append(user_2_data['artists_long_term']))


# user_1_tracks_ids = []
# user_1_artists_ids = []
# user_2_tracks_ids = []
# user_2_artists_ids = []

for track in user_1_combined['tracks']:
  user_1_tracks_ids.append(track['id'])

for artist in user_1_combined['artists']:
  user_1_artists_ids.append(artist['id'])


for track in user_2_combined['tracks']:
  user_2_tracks_ids.append(track['id'])

for artist in user_2_combined['artists']:
  user_2_artists_ids.append(artist['id'])



In [50]:
user_1_tracks_ids = list(set(user_1_tracks_ids))
user_1_artists_ids = list(set(user_1_artists_ids))
user_2_tracks_ids = list(set(user_2_tracks_ids))
user_2_artists_ids = list(set(user_2_artists_ids))

print(user_1_tracks_ids)

['50P6DaCmJFDRbb9wvbCP4i', '48HzNPszVrLIyR256ZDpUF', '4dRNNBipJ6wwSEEwa4FfcY', '5EZhOlmGIuo9R3wXrcZwip', '0cBohovCcJEFKOdEsw2g5F', '0ICNCIl4m3g4xC8zvxCq5f', '0fBwXotsWjsKPlWvwm4BML', '5aByG0Jfx5eCng6xM1Nbrt', '0kZynK7V8w689TJzxBqFcA', '35nAsRTaca9rKZNxoPH0TD', '0FwVoUXnvbJgxKtWjkmM2w', '5kqgGpu9KPDAie8L0cb71i', '6MDBkIEBEodrBPG5CPb8IG', '0rY6eMNP3hXxEodeEOdKIk', '0TmdITeqXgJidq62RoJKey', '7kX56S5T5acyvFmEWwDait', '0rvMMD5uFW0lzAW8EPp5yU', '3XptdHIz8JT9nTGhIkpBVn', '6kYZXHza47jqW78T17U3G0', '3L98CBzSkm9h2AaXN9Q3K6', '4XJtyopZ8I3k1ZwAxX02RK', '1UJWs2dI3DlFj28TfTMqla', '6FcCQcDDfWx6q5oPl2f7Bw', '5sdgUBxKR6XKlryKwwulwZ', '2BQMWJjScSxJb27jgDYH2Z', '3O9mhTBcaVWBUd19CArxFM', '7En0CVHGFnJrUNRUwMZgAX', '0ZsMCUijUwA6Hvv7tAwKJh', '6koWevx9MqN6efQ6qreIbm', '0poNIN6WkdhkR6pEpQhWVw', '4J1Ie6FTecIid0yPEHNGST', '1kIYdvuLyGs0dBpMTgU2Ab', '6gIup5MOcUOaBRPZYilVAp', '6HguG9HRb1Ke1bhihfE4m8', '4Ox8eT8WSOZNa72Te0qN2t', '4YPYvh5tcfQ8iMaKd6pGE8', '5zJWRDohBVAB1D4X41EIQ0', '0UMC6zmuw4N7uSFJerVDHy', '0jdaHGaVsa

In [51]:
common_tracks_ids = list(set(user_1_tracks_ids).intersection(set(user_2_tracks_ids)))
common_artists_ids = list(set(user_1_artists_ids).intersection(set(user_2_artists_ids)))
user_1_artists_ids_unique = list(set(user_1_artists_ids).difference(set(user_2_artists_ids)))
user_2_artists_ids_unique = list(set(user_2_artists_ids).difference(set(user_1_artists_ids)))

print(common_tracks_ids)
print(common_artists_ids)

['6HguG9HRb1Ke1bhihfE4m8']
['4O15NlyKLIASxsJ0PrXPfz', '7CajNmpbOovFoOoasH2HaY', '1t20wYnTiAT0Bs7H1hv9Wt', '4MCBfE4596Uoi2O4DtmEMz', '5K4W6rqBFWDnAN6FQUkS6x', '23fqKkggKUBHNkbKtXEls4', '2qxJFvFYMEDqd7ui6kSAcq', '3mIj9lX2MWuHmhNCA7LSCW', '5JZ7CnR6gTvEMKX4g70Amv']


In [53]:
common_tracks = []
common_artists = []

for track_id in common_tracks_ids:
  for track in user_1_combined['tracks']:
    if track_id == track['id']:
      common_tracks.append(track)
      break

for artist_id in common_artists_ids:
  for artist in user_2_combined['artists']:
    if artist_id == artist['id']:
      common_artists.append(artist)
      break

In [54]:
common_artists

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/4O15NlyKLIASxsJ0PrXPfz'},
  'followers': {'href': None, 'total': 8919705},
  'genres': ['melodic rap', 'philly rap', 'rap', 'trap'],
  'href': 'https://api.spotify.com/v1/artists/4O15NlyKLIASxsJ0PrXPfz',
  'id': '4O15NlyKLIASxsJ0PrXPfz',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/271c672ac7a84a95f3c557177a29b8936923d152',
    'width': 640},
   {'height': 320,
    'url': 'https://i.scdn.co/image/0b57d28b204187ca97d4dbd700fa7e347ff05fa2',
    'width': 320},
   {'height': 160,
    'url': 'https://i.scdn.co/image/43fb40412c6dd226997c52e3647304a97b1709a9',
    'width': 160}],
  'name': 'Lil Uzi Vert',
  'popularity': 94,
  'rank': 22,
  'type': 'artist',
  'uri': 'spotify:artist:4O15NlyKLIASxsJ0PrXPfz',
  'weight': 0.26333567909922584},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/7CajNmpbOovFoOoasH2HaY'},
  'followers': {'href': None, 'total': 20008537},
  'genres': ['dance pop',
   'ed

In [55]:
#names of the artists we have in common
for artist in common_artists:
  print(artist['name'])

Lil Uzi Vert
Calvin Harris
EDEN
Juice WRLD
Kanye West
Kygo
Zedd
The 1975
Lauv


In [56]:
#looking for what time frame each of these artists that we share come from
common_artists_time_frame_data = []

for artist in common_artists:
  artist_data = {}
  artist_data['id'] = artist['id']
  artist_data['name'] = artist['name']
  user_1_time_frame_data = []
  user_2_time_frame_data = []
  for time_frame in indices['time_frames']:
    index_string = 'artists_' + time_frame
    for user_1_artist in user_1_data[index_string]:
      if artist['name'] == user_1_artist['name']:
        user_1_time_frame_data.append(time_frame)
        break
    for user_2_artist in user_2_data[index_string]:
      if artist['name'] == user_2_artist['name']:
        user_2_time_frame_data.append(time_frame)
        break
  artist_data['user_1_time_frames'] = user_1_time_frame_data
  artist_data['user_2_time_frames'] = user_2_time_frame_data
  common_artists_time_frame_data.append(artist_data)

In [57]:
common_artists_time_frame_data 

[{'id': '4O15NlyKLIASxsJ0PrXPfz',
  'name': 'Lil Uzi Vert',
  'user_1_time_frames': ['medium_term', 'long_term'],
  'user_2_time_frames': ['short_term', 'medium_term']},
 {'id': '7CajNmpbOovFoOoasH2HaY',
  'name': 'Calvin Harris',
  'user_1_time_frames': ['long_term'],
  'user_2_time_frames': ['medium_term', 'long_term']},
 {'id': '1t20wYnTiAT0Bs7H1hv9Wt',
  'name': 'EDEN',
  'user_1_time_frames': ['short_term', 'long_term'],
  'user_2_time_frames': ['short_term', 'medium_term', 'long_term']},
 {'id': '4MCBfE4596Uoi2O4DtmEMz',
  'name': 'Juice WRLD',
  'user_1_time_frames': ['long_term'],
  'user_2_time_frames': ['medium_term']},
 {'id': '5K4W6rqBFWDnAN6FQUkS6x',
  'name': 'Kanye West',
  'user_1_time_frames': ['medium_term', 'long_term'],
  'user_2_time_frames': ['short_term', 'medium_term', 'long_term']},
 {'id': '23fqKkggKUBHNkbKtXEls4',
  'name': 'Kygo',
  'user_1_time_frames': ['long_term'],
  'user_2_time_frames': ['short_term', 'medium_term']},
 {'id': '2qxJFvFYMEDqd7ui6kSAcq',


In [58]:
#looking for what time frame each of these artists that we share come from, and then what position they are in that time period
#a value of 0 means that they were not in the list
common_artists_time_frame_data_numeric = []

for artist in common_artists:
  artist_data = {}
  artist_data['id'] = artist['id']
  artist_data['name'] = artist['name']
  user_1_time_frame_data = {}
  user_2_time_frame_data = {}
  for time_frame in indices['time_frames']:
    index_string = 'artists_' + time_frame
    user_1_time_frame_data[time_frame + '_total'] = len(user_1_data[index_string])
    rank = 0
    count = 0
    for user_1_artist in user_1_data[index_string]:
      count += 1
      if artist['name'] == user_1_artist['name']:
        rank = count
        break
    user_1_time_frame_data[time_frame + '_rank'] = rank
    user_2_time_frame_data[time_frame + '_total'] = len(user_2_data[index_string])
    rank = 0
    count = 0
    for user_2_artist in user_2_data[index_string]:
      count +=1
      if artist['name'] == user_2_artist['name']:
        rank = count
        break
    user_2_time_frame_data[time_frame + '_rank'] = rank
  artist_data['user_1_time_data'] = user_1_time_frame_data
  artist_data['user_2_time_data'] = user_2_time_frame_data
  common_artists_time_frame_data_numeric.append(artist_data)

In [59]:
common_artists_time_frame_data_numeric

[{'id': '4O15NlyKLIASxsJ0PrXPfz',
  'name': 'Lil Uzi Vert',
  'user_1_time_data': {'long_term_rank': 5,
   'long_term_total': 50,
   'medium_term_rank': 43,
   'medium_term_total': 50,
   'short_term_rank': 0,
   'short_term_total': 47},
  'user_2_time_data': {'long_term_rank': 0,
   'long_term_total': 50,
   'medium_term_rank': 48,
   'medium_term_total': 50,
   'short_term_rank': 22,
   'short_term_total': 30}},
 {'id': '7CajNmpbOovFoOoasH2HaY',
  'name': 'Calvin Harris',
  'user_1_time_data': {'long_term_rank': 42,
   'long_term_total': 50,
   'medium_term_rank': 0,
   'medium_term_total': 50,
   'short_term_rank': 0,
   'short_term_total': 47},
  'user_2_time_data': {'long_term_rank': 34,
   'long_term_total': 50,
   'medium_term_rank': 30,
   'medium_term_total': 50,
   'short_term_rank': 0,
   'short_term_total': 30}},
 {'id': '1t20wYnTiAT0Bs7H1hv9Wt',
  'name': 'EDEN',
  'user_1_time_data': {'long_term_rank': 22,
   'long_term_total': 50,
   'medium_term_rank': 0,
   'medium_ter

In [60]:
# user_1_artists_unique = []
# user_2_artists_unique = []

# for artist_id in user_1_artists_ids_unique:
#   for artist in user_1_artists_all:
#     if artist_id == artist['id']:
#       user_1_artists_unique.append(artist)
#       break

# for artist_id in user_2_artists_ids_unique:
#   for artist in user_2_artists_all:
#     if artist_id == artist['id']:
#       user_2_artists_unique.append(artist)
#       break

# print(user_1_artists_unique)
# print(user_2_artists_unique)

def combine2UsersData(user_1_combined, user_2_combined):
  # will return aggregate, common, user_1_unique, and user_2_unique
  aggregate_data = {'artists': [], 'tracks': []}
  common_data = {'artists': [], 'tracks': []}
  user_1_unique_data = {'artists': [], 'tracks': []}
  user_2_unique_data = {'artists': [], 'tracks': []}


  for resource_type in indices['resource_types']:
    combined_data = []
    for resource in user_1_combined[resource_type]:
      combined_data.append(resource.copy())
    
    redundant_data = []
    for combined_resource in combined_data:
      for resource in user_2_combined[resource_type]:
        if combined_resource['id'] == resource['id']:
          combined_resource['weight'] += resource['weight']
          redundant_data.append(combined_resource.copy())
      
    user_2_data = []
    for resource in user_2_combined[resource_type]:
      to_add = True
      for redundant_resource in redundant_data:
        if resource['id'] == redundant_resource['id']:
          to_add = False
      if to_add:
        combined_data.append(resource.copy())
        user_2_data.append(resource.copy())
      
    user_1_data = []
    for resource in user_1_combined[resource_type]:
      to_add = True
      for redundant_resource in redundant_data:
        if resource['id'] == redundant_resource['id']:
          to_add = False
      if to_add:
        user_1_data.append(resource.copy())

    aggregate_data[resource_type] = combined_data
    common_data[resource_type] = redundant_data
    user_1_unique_data[resource_type] = user_1_data
    user_2_unique_data[resource_type] = user_2_data

  return aggregate_data, common_data, user_1_unique_data, user_2_unique_data


aggregate, common, user_1_unique, user_2_unique = combine2UsersData(user_1_combined, user_2_combined)

print(len(common['artists']))
for artist in common['artists']:
  print(artist['name'] + ' : ' + str(artist['weight']))

print('============================================')

print(len(aggregate['artists']))
for artist in aggregate['artists']:
  print(artist['name'] + ' : ' + str(artist['weight']))

print('============================================')

print(len(user_1_unique['artists']))
for artist in user_1_unique['artists']:
  print(artist['name'] + ' : ' + str(artist['weight']))

print('============================================')

print(len(user_2_unique['artists']))
for artist in user_2_unique['artists']:
  print(artist['name'] + ' : ' + str(artist['weight']))

print('============================================')



9
Lil Uzi Vert : 0.7701384001876612
Kanye West : 1.3413323950269764
EDEN : 1.100536466460647
Zedd : 0.7272108843537415
The 1975 : 1.3876612714051137
Lauv : 1.2433497536945812
Kygo : 0.612291813277035
Calvin Harris : 0.5789115646258503
Juice WRLD : 0.3408163265306122
164
Dance Gavin Dance : 0.9662821650399289
Pulses. : 0.8937000887311446
Delta Sleep : 0.8686779059449867
Owane : 0.8384797397219759
Origami Button : 0.7945578231292516
Chon : 0.7694173321502513
TTNG : 0.7684708666075125
Meliorist : 0.7382727003845015
Properties of Nature : 0.7303460514640638
Intervals : 0.7095829636202307
Sleep Token : 0.6639160011830818
Porter Robinson : 0.6333333333333333
Tyler, The Creator : 0.6129251700680272
Eidola : 0.5583850931677018
Flume : 0.5299319727891156
Polyphia : 0.5140490979000296
Daft Punk : 0.5136054421768708
Lil Uzi Vert : 0.7701384001876612
Kanye West : 1.3413323950269764
Wolf & Bear : 0.50150842945874
Floral : 0.49174800354924575
Invalids : 0.4843833185448092
Jank : 0.47332150251404914


In [61]:
aggregate_artists = aggregate['artists']
common_artists = common['artists']
user_1_artists_unique = user_1_unique['artists']
user_2_artists_unique = user_2_unique['artists']

In [62]:
user_1_artists_unique_stripped = []
user_2_artists_unique_stripped = []
common_artists_stripped = []

for artist in user_1_artists_unique:
  user_1_artists_unique_stripped.append({'name': artist['name'], 'id': artist['id'], 'weight': artist['weight']})

for artist in user_2_artists_unique:
  user_2_artists_unique_stripped.append({'name': artist['name'], 'id': artist['id'], 'weight': artist['weight']})

for artist in common_artists:
  common_artists_stripped.append({'name': artist['name'], 'id': artist['id'], 'weight': artist['weight']})

for artist in user_1_artists_unique_stripped:
  print(artist)

print('\n=====================================================================\n')
for artist in user_2_artists_unique_stripped:
  print(artist)

print('\n=====================================================================\n')

for artist in common_artists_stripped:
  print(artist)

print('\n=====================================================================\n')

{'name': 'Dance Gavin Dance', 'id': '6guC9FqvlVboSKTI77NG2k', 'weight': 0.9662821650399289}
{'name': 'Pulses.', 'id': '0tko2stnUOqejIto8rKRh7', 'weight': 0.8937000887311446}
{'name': 'Delta Sleep', 'id': '05MlomiA9La0OiNIAGqECk', 'weight': 0.8686779059449867}
{'name': 'Owane', 'id': '2LkMP01m9PIf5r87iC0HI2', 'weight': 0.8384797397219759}
{'name': 'Origami Button', 'id': '36PWgrlKrQpL8vvRR2kURR', 'weight': 0.7945578231292516}
{'name': 'Chon', 'id': '2JFljHPanIjYy2QqfNYvC0', 'weight': 0.7694173321502513}
{'name': 'TTNG', 'id': '5TUa95aB5Vu2CzwCnZd6t0', 'weight': 0.7684708666075125}
{'name': 'Meliorist', 'id': '7DwHQdzAPFLx6Gug80MjU1', 'weight': 0.7382727003845015}
{'name': 'Properties of Nature', 'id': '1HIPy5eD1xnDTSKjIzzlcU', 'weight': 0.7303460514640638}
{'name': 'Intervals', 'id': '0xpJGyjbEzkWSNfcf2tcMl', 'weight': 0.7095829636202307}
{'name': 'Sleep Token', 'id': '2n2RSaZqBuUUukhbLlpnE6', 'weight': 0.6639160011830818}
{'name': 'Porter Robinson', 'id': '3dz0NnIZhtKKeXZxLOxCam', 'wei

Getting a lot of data at once, so this may take a while

In [66]:
for artist in user_1_artists_unique_stripped:
  related_artist_ids = []
  time.sleep(0.1)
  similar_artists = spotify.generic_get('https://api.spotify.com/v1/artists/' + artist['id'] +'/related-artists')
  for similar_artist in similar_artists['artists']:
    related_artist_ids.append({'name':similar_artist['name'],'id':similar_artist['id']})
  artist['similar_artists'] = related_artist_ids

for artist in user_2_artists_unique_stripped:
  related_artist_ids = []
  time.sleep(0.1)
  similar_artists = spotify.generic_get('https://api.spotify.com/v1/artists/' + artist['id'] +'/related-artists')
  for similar_artist in similar_artists['artists']:
    related_artist_ids.append({'name':similar_artist['name'],'id':similar_artist['id']})
  artist['similar_artists'] = related_artist_ids

for artist in common_artists_stripped:
  related_artist_ids = []
  time.sleep(0.1)
  similar_artists = spotify.generic_get('https://api.spotify.com/v1/artists/' + artist['id'] +'/related-artists')
  for similar_artist in similar_artists['artists']:
    related_artist_ids.append({'name':similar_artist['name'],'id':similar_artist['id']})
  artist['similar_artists'] = related_artist_ids



for artist in user_1_artists_unique_stripped:
  print(artist)

print('\n=====================================================================\n')
for artist in user_2_artists_unique_stripped:
  print(artist)

print('\n=====================================================================\n')

for artist in common_artists_stripped:
  print(artist)

print('\n=====================================================================\n')

{'name': 'Dance Gavin Dance', 'id': '6guC9FqvlVboSKTI77NG2k', 'weight': 0.9662821650399289, 'similar_artists': [{'name': 'Emarosa', 'id': '68tKVjVvcqUfKFFLr2j0Ek'}, {'name': 'Isles & Glaciers', 'id': '6dHr8Y0nPrnHiBOuJzuu8F'}, {'name': 'Chiodos', 'id': '4ZIBrskTJWmkJUvM7uAu1o'}, {'name': 'Scary Kids Scaring Kids', 'id': '3cdos5vq3wSUR8mEvMiqq9'}, {'name': 'Jonny Craig', 'id': '46MmlFKM8ivDsUnGOejvSM'}, {'name': 'Underoath', 'id': '3GzWhE2xadJiW8MqRKIVSK'}, {'name': 'I See Stars', 'id': '3iCJOi5YKh247eutgCyLFe'}, {'name': 'Saosin', 'id': '1NUOfvAhA9AvsF1ISMkgHX'}, {'name': 'I Set My Friends On Fire', 'id': '2QF7jwUc5zgjSbOTYh1xV0'}, {'name': 'From First To Last', 'id': '5n71fmAB8Un2IfAPn64erN'}, {'name': 'blessthefall', 'id': '7t2C8WwLyKUKRe0LVh8zl9'}, {'name': 'A Skylit Drive', 'id': '2HSp2S2KwxKE7sASl6bMR9'}, {'name': 'Memphis May Fire', 'id': '7cNNNhdJDrt3vgQjwSavNf'}, {'name': 'Circa Survive', 'id': '11FY888Qctoy6YueCpFkXT'}, {'name': 'The Fall of Troy', 'id': '5fuQrhMRYMtoO9uOlFad4

In [67]:
artist_map = []

for artist in common_artists_stripped + user_1_artists_unique_stripped + user_2_artists_unique_stripped:
  item = {'name': artist['name'],  'id': artist['id'], 'weight': artist['weight']}
  edges = []
  for similar_artist in artist['similar_artists']:
    for artist_2 in common_artists_stripped + user_1_artists_unique_stripped + user_2_artists_unique_stripped:
      # print(artist)
      # print(similar_artist)
      # print('----------------------------------------------')
      # print(f'{artist_id} ?= {sim_artist_id}')
      # print(artist['name'] + ' ?= ' + artist_2['name'] + '.' + similar_artist['name'])
      if artist_2['id'] == similar_artist['id']:
        edges.append({'name': artist_2['name'], 'id':artist_2['id'], 'weight': artist['weight']})
  item['edges'] = edges
  artist_map.append(item)

artist_map

[{'edges': [{'id': '50co4Is1HCEo8bhOyUWKpn',
    'name': 'Young Thug',
    'weight': 0.7701384001876612},
   {'id': '6oMuImdp5ZcFhWP0ESe6mG',
    'name': 'Migos',
    'weight': 0.7701384001876612}],
  'id': '4O15NlyKLIASxsJ0PrXPfz',
  'name': 'Lil Uzi Vert',
  'weight': 0.7701384001876612},
 {'edges': [{'id': '2YZyLoL8N0Wb9xBt1NhZWg',
    'name': 'Kendrick Lamar',
    'weight': 1.3413323950269764},
   {'id': '1anyVhU62p31KFi8MEzkbf',
    'name': 'Chance the Rapper',
    'weight': 1.3413323950269764},
   {'id': '73sIBHcqh3Z3NyqHKZ7FOL',
    'name': 'Childish Gambino',
    'weight': 1.3413323950269764}],
  'id': '5K4W6rqBFWDnAN6FQUkS6x',
  'name': 'Kanye West',
  'weight': 1.3413323950269764},
 {'edges': [{'id': '50JJSqHUf2RQ9xsHs0KMHg',
    'name': 'Jon Bellion',
    'weight': 1.100536466460647}],
  'id': '1t20wYnTiAT0Bs7H1hv9Wt',
  'name': 'EDEN',
  'weight': 1.100536466460647},
 {'edges': [{'id': '7DMveApC7UnC2NPfPvlHSU',
    'name': 'Cheat Codes',
    'weight': 0.7272108843537415},
 

In [68]:
import networkx as nx
G=nx.Graph()
G

In [69]:
for artist in artist_map:
  G.add_node(artist['name'])

print(G.nodes())

print(len(G.nodes()))

['Lil Uzi Vert', 'Kanye West', 'EDEN', 'Zedd', 'The 1975', 'Lauv', 'Kygo', 'Calvin Harris', 'Juice WRLD', 'Dance Gavin Dance', 'Pulses.', 'Delta Sleep', 'Owane', 'Origami Button', 'Chon', 'TTNG', 'Meliorist', 'Properties of Nature', 'Intervals', 'Sleep Token', 'Porter Robinson', 'Tyler, The Creator', 'Eidola', 'Flume', 'Polyphia', 'Daft Punk', 'Wolf & Bear', 'Floral', 'Invalids', 'Jank', 'Good Game', 'Eternity Forever', 'Covet', 'Deorro', 'Ben Rosett', 'Lil Wayne', 'Jon Bellion', 'flor', 'The Band CAMINO', 'deadmau5', 'Tycho', 'Michiru Oshima', 'Animals As Leaders', 'Catbamboo', 'Andrew Bayer', 'Louis The Child', 'Resilia', 'Tensor', 'Their Dogs Were Astronauts', 'Colorblind', 'Sea in the Sky', 'Sawce', 'Anemoria', 'Pentagon', 'Benal', 'Rustycore', 'Clairo', 'Walk Off the Earth', 'Logic', 'Guitar Fight from Fooly Cooly', 'WILLOW', 'Totorro', 'Kyle Dixon & Michael Stein', 'Surfaces', "Panucci's Pizza", 'Valleyheart', 'Snakedersch', 'Jacob Collier', 'Land Wars', 'Galleons', 'Hugo Helmig'

In [70]:
for artist in artist_map:
  for edge in artist['edges']:
    graph_edge = (artist['name'], edge['name'])
    G.add_edge(*graph_edge)

print(G.edges())
len(G.edges())

[('Lil Uzi Vert', 'Young Thug'), ('Lil Uzi Vert', 'Migos'), ('Lil Uzi Vert', 'Travis Scott'), ('Kanye West', 'Kendrick Lamar'), ('Kanye West', 'Chance the Rapper'), ('Kanye West', 'Childish Gambino'), ('EDEN', 'Jon Bellion'), ('Zedd', 'Cheat Codes'), ('Zedd', 'Galantis'), ('Zedd', 'Gryffin'), ('Zedd', 'Calvin Harris'), ('Zedd', 'The Chainsmokers'), ('Zedd', 'Maroon 5'), ('The 1975', 'The Japanese House'), ('The 1975', 'The Band CAMINO'), ('The 1975', 'flor'), ('The 1975', 'The Neighbourhood'), ('Lauv', 'Bazzi'), ('Kygo', 'Cheat Codes'), ('Kygo', 'Gryffin'), ('Kygo', 'The Chainsmokers'), ('Kygo', 'Galantis'), ('Kygo', 'Calvin Harris'), ('Kygo', 'Marshmello'), ('Calvin Harris', 'The Chainsmokers'), ('Calvin Harris', 'Cheat Codes'), ('Calvin Harris', 'Galantis'), ('Juice WRLD', 'Young Thug'), ('Dance Gavin Dance', 'The Fall of Troy'), ('Pulses.', 'AnimalJam'), ('Pulses.', 'Wolf & Bear'), ('Pulses.', 'Properties of Nature'), ('Pulses.', 'Anemoria'), ('Pulses.', 'Origami Button'), ('Pulses.

274

In [71]:
# nodes

user_1_node_list = []
user_2_node_list = []
common_node_list = []

user_1_node_sizes = []
user_2_node_sizes = []
common_node_sizes = []

for artist in user_1_artists_unique_stripped:
  user_1_node_list.append(artist['name'])
  user_1_node_sizes.append(artist['weight'] * 2000)

for artist in user_2_artists_unique_stripped:
  user_2_node_list.append(artist['name'])
  user_2_node_sizes.append(artist['weight'] * 2000)

for artist in common_artists_stripped:
  common_node_list.append(artist['name'])
  common_node_sizes.append(artist['weight'] * 2000)

user_1_node_dict = {}
user_2_node_dict = {}
common_node_dict = {}

for label in user_1_node_list:
  user_1_node_dict[label] = label 

for label in user_2_node_list:
  user_2_node_dict[label] = label 

for label in common_node_list:
  common_node_dict[label] = label 

print(common_node_sizes)
print(common_node_list)
print(user_1_node_list)
print(user_2_node_list)

[1540.2768003753224, 2682.664790053953, 2201.072932921294, 1454.421768707483, 2775.3225428102273, 2486.6995073891626, 1224.58362655407, 1157.8231292517005, 681.6326530612245]
['Lil Uzi Vert', 'Kanye West', 'EDEN', 'Zedd', 'The 1975', 'Lauv', 'Kygo', 'Calvin Harris', 'Juice WRLD']
['Dance Gavin Dance', 'Pulses.', 'Delta Sleep', 'Owane', 'Origami Button', 'Chon', 'TTNG', 'Meliorist', 'Properties of Nature', 'Intervals', 'Sleep Token', 'Porter Robinson', 'Tyler, The Creator', 'Eidola', 'Flume', 'Polyphia', 'Daft Punk', 'Wolf & Bear', 'Floral', 'Invalids', 'Jank', 'Good Game', 'Eternity Forever', 'Covet', 'Deorro', 'Ben Rosett', 'Lil Wayne', 'Jon Bellion', 'flor', 'The Band CAMINO', 'deadmau5', 'Tycho', 'Michiru Oshima', 'Animals As Leaders', 'Catbamboo', 'Andrew Bayer', 'Louis The Child', 'Resilia', 'Tensor', 'Their Dogs Were Astronauts', 'Colorblind', 'Sea in the Sky', 'Sawce', 'Anemoria', 'Pentagon', 'Benal', 'Rustycore', 'Clairo', 'Walk Off the Earth', 'Logic', 'Guitar Fight from Fooly

In [72]:
G.nodes

NodeView(('Lil Uzi Vert', 'Kanye West', 'EDEN', 'Zedd', 'The 1975', 'Lauv', 'Kygo', 'Calvin Harris', 'Juice WRLD', 'Dance Gavin Dance', 'Pulses.', 'Delta Sleep', 'Owane', 'Origami Button', 'Chon', 'TTNG', 'Meliorist', 'Properties of Nature', 'Intervals', 'Sleep Token', 'Porter Robinson', 'Tyler, The Creator', 'Eidola', 'Flume', 'Polyphia', 'Daft Punk', 'Wolf & Bear', 'Floral', 'Invalids', 'Jank', 'Good Game', 'Eternity Forever', 'Covet', 'Deorro', 'Ben Rosett', 'Lil Wayne', 'Jon Bellion', 'flor', 'The Band CAMINO', 'deadmau5', 'Tycho', 'Michiru Oshima', 'Animals As Leaders', 'Catbamboo', 'Andrew Bayer', 'Louis The Child', 'Resilia', 'Tensor', 'Their Dogs Were Astronauts', 'Colorblind', 'Sea in the Sky', 'Sawce', 'Anemoria', 'Pentagon', 'Benal', 'Rustycore', 'Clairo', 'Walk Off the Earth', 'Logic', 'Guitar Fight from Fooly Cooly', 'WILLOW', 'Totorro', 'Kyle Dixon & Michael Stein', 'Surfaces', "Panucci's Pizza", 'Valleyheart', 'Snakedersch', 'Jacob Collier', 'Land Wars', 'Galleons', 'Hug

In [75]:

# pos=nx.nx.nx_pydot.graphviz_layout(G)
# pos=nx.nx.nx_pydot.pydot_layout(G)

pos = nx.drawing.layout.spring_layout(G)

plt.figure()

nx.draw_networkx_nodes(G,pos,
                       nodelist=common_node_list,
                       node_color='purple',
                       node_size = common_node_sizes,
                       with_labels = True,
                   alpha=0.3)
nx.draw_networkx_labels(G, pos, labels=common_node_dict)

nx.draw_networkx_nodes(G,pos,
                       nodelist=user_1_node_list,
                       node_color='lightblue',
                       node_size = user_1_node_sizes,
                       with_labels = True,
                   alpha=0.8)
nx.draw_networkx_labels(G, pos, labels=user_1_node_dict)

nx.draw_networkx_nodes(G,pos,
                       nodelist=user_2_node_list,
                       node_color='pink',
                       node_size = user_2_node_sizes,
                       with_labels = True,
                   alpha=0.8)
nx.draw_networkx_labels(G, pos, labels=user_2_node_dict)



nx.draw_networkx_edges(G,pos,alpha=0.5)

# nx.draw_networkx_nodes(G,pos,
#                        nodelist=user_2_node_list,
#                        node_color='r',
#                        with_labels = True,
#                    alpha=0.8)

# nx.draw_networkx_nodes(G,pos,
#                        nodelist=common_node_list,
#                        node_color='purple',
#                        with_labels = True,
#                    alpha=0.8)

# nx.draw_networkx_edges(G,pos,alpha=0.5)
# # nx.draw(G,pos = nx.nx_pydot.graphviz_layout(G), with_labels = True,node_color='lightblue', edge_color='gray')

# plt.rcParams["figure.figsize"] = (40,20)
# # plt.savefig("avery_medium_term.png") # save as png
# plt.show() # display

# nx.draw(G,pos = nx.nx_pydot.graphviz_layout(G), with_labels = True,node_color='lightblue', edge_color='gray')

plt.rcParams["figure.figsize"] = (60,30)
# plt.savefig("avery_medium_term.png") # save as png
plt.show() # display

TypeError: ignored

<Figure size 432x288 with 0 Axes>